# Première classification

L'objectif ici est d'utiliser un premier classifier. Pour commencer nous prendrons SVM qui obtient souvent de bons résultats sur les données textuelles.

Nous pouvons donc, pour simplifier, créer un jeu d'apprentissage et un jeu de test et évaluer le résultat d'un classifieur SVM placé dans un pipeline.

In [11]:
from pretraitement import TextPreTraitement
from clean import clean_claimKG

import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [2]:
file_name = "../data/claimKG.csv"
origin = pd.read_csv(file_name)

In [4]:
kg = origin.copy()

kg = clean_claimKG(kg, verbose=False, inplace=True)

Pour un premier test, nous allons prendre comme `X` les assertions et `Y` les valeurs de véracité.

Note: les valeurs de véracité ne sont pour l'instant pas disponible, à voir ou les trouver.

Donc, pour l'instant, on ne garde que les lignes qu'on peut exploiter.

In [14]:
def translate(rating):
    t = {'false': 0,
        'true': 1,
        'mixture': 2,
        'other': -1,
        'mostly true': 2,
        'mostly false': 2,
        'half-true': 2,
    }
    return t[rating]
    
# mettre toutes les valeurs en miniscule
kg['rating_alternateName'] = kg['rating_alternateName'].apply(lambda x : str(x).lower())

keep = ['false', 'true', 'mostly false', 'mostly true','half-true', 'mixture']
keep_lst = kg.index[kg['rating_alternateName'].isin(keep)].tolist()
# print(kg.loc[keep_lst]['rating_alternateName'].value_counts())

rm_lst = kg.index[~kg['rating_alternateName'].isin(keep)].tolist()
kg.drop(rm_lst, inplace=True)
kg['ratingValue'] = kg['rating_alternateName'].apply(lambda x: translate(x))
print(kg['ratingValue'].value_counts())
del rm_lst, keep_lst, translate

0    10701
2    10258
1     4584
Name: ratingValue, dtype: int64


Création d'un jeu d'apprentissage et de test

In [6]:
X=kg['claimReview_claimReviewed']
y=kg['ratingValue']

X_train,X_test,y_train,y_test=train_test_split(X, y, 
                                               train_size=0.7 ,
                                               random_state=30)

In [8]:
vectorizer = TfidfVectorizer(
            lowercase=False,
            ngram_range=(2,2),
            tokenizer=None,
            preprocessor=lambda x:x,
            min_df=0.01, 
            max_df=0.9)

Création du pipeline

In [10]:
pipe = Pipeline([("cleaner", TextPreTraitement(stopword=True)),
                 ("count_vectorizer", vectorizer),
                 ("SVM", SVC())])
pipe.fit(X_train,y_train)

print("pipeline créé")

pipeline créé


On teste le modèle

In [12]:
y_pred = pipe.predict(X_test)

print("train accuracy:",accuracy_score(y_train, pipe.predict(X_train)))
print("test accuracy:",accuracy_score(y_test, y_pred))


train accuracy: 0.4261185682326622
test accuracy: 0.438862064465614
